In [1]:
import numpy as np
import pandas as pd
from bert4keras.backend import keras, set_gelu, search_layer, K
from bert4keras.tokenizers import Tokenizer
from bert4keras.models import build_transformer_model
from bert4keras.optimizers import Adam, extend_with_piecewise_linear_lr
from bert4keras.snippets import sequence_padding

from keras.layers import Lambda, Dense, Dropout
from keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
import tensorflow as tf
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

config=tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.24
sess=tf.compat.v1.Session(config=config)

2021-10-25 20:10:15.404659: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2021-10-25 20:10:15.424162: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2021-10-25 20:10:15.427207: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x556ca3e8b6b0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-10-25 20:10:15.427290: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-10-25 20:10:15.430561: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-10-25 20:10:15.605369: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x556ca44bae70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2021-10-25 20:10:15.605473: I tensorflow/compil

In [3]:
config_path = 'RoBERTa-tiny3L312-clue/bert_config.json'
checkpoint_path = 'RoBERTa-tiny3L312-clue/bert_model.ckpt'
dict_path = 'RoBERTa-tiny3L312-clue/vocab.txt'

In [2]:
train_data = pd.read_csv('data/sentiment_analysis_trainingset.csv').iloc[:, 1:].dropna()
valid_data = pd.read_csv('data/sentiment_analysis_validationset.csv').iloc[:, 1:].dropna()

In [5]:
train_data.head()

,content,location_traffic_convenience,location_distance_from_business_district,location_easy_to_find,service_wait_time,service_waiters_attitude,service_parking_convenience,service_serving_speed,price_level,price_cost_effective,...,environment_decoration,environment_noise,environment_space,environment_cleaness,dish_portion,dish_taste,dish_look,dish_recommendation,others_overall_experience,others_willing_to_consume_again
0,"""吼吼吼，萌死人的棒棒糖，中了大众点评的霸王餐，太可爱了。一直就好奇这个棒棒糖是怎么个东西，...",-2,-2,-2,-2,1,-2,-2,-2,-2,...,-2,-2,-2,-2,-2,-2,1,-2,1,-2
1,"""第三次参加大众点评网霸王餐的活动。这家店给人整体感觉一般。首先环境只能算中等，其次霸王餐提...",-2,-2,-2,-2,-2,-2,-2,0,-2,...,0,0,0,0,1,-2,-2,-2,1,-2
2,"""4人同行 点了10个小吃\n榴莲酥 榴莲味道不足 松软 奶味浓\n虾饺 好吃 两颗大虾仁\...",-2,-2,-2,-2,0,-2,1,0,-2,...,-2,-2,1,-2,0,1,-2,-2,0,-2
3,"""之前评价了莫名其妙被删 果断继续差评！ 换了菜单 价格更低 开始砸牌子 但套餐还是有150...",-2,-2,-2,-2,-2,-2,-2,0,-2,...,-2,-2,-2,-2,-2,-1,-2,-2,-1,-1
4,"""出乎意料地惊艳，椰子鸡清热降火，美容养颜，大大满足了爱吃火锅怕上火星人。椰子冻是帅帅的老板...",-2,-2,-2,-2,-2,-2,-2,-2,-2,...,-2,-2,-2,-2,-2,1,1,-2,1,-2


In [5]:
train_data.iloc[:, 1:] = train_data.iloc[:, 1:].apply(lambda x: x + 2)
valid_data.iloc[:, 1:] = valid_data.iloc[:, 1:].apply(lambda x: x + 2)

In [6]:
train_data.columns

Index(['content', 'location_traffic_convenience',
       'location_distance_from_business_district', 'location_easy_to_find',
       'service_wait_time', 'service_waiters_attitude',
       'service_parking_convenience', 'service_serving_speed', 'price_level',
       'price_cost_effective', 'price_discount', 'environment_decoration',
       'environment_noise', 'environment_space', 'environment_cleaness',
       'dish_portion', 'dish_taste', 'dish_look', 'dish_recommendation',
       'others_overall_experience', 'others_willing_to_consume_again'],
      dtype='object')

In [7]:
# for column in train_data.columns:
#     print(valid_data[column].value_counts())

In [8]:
len(train_data)

105000

In [9]:
len(valid_data)

15000

In [10]:
tags = train_data.columns[1:]
tags

Index(['location_traffic_convenience',
       'location_distance_from_business_district', 'location_easy_to_find',
       'service_wait_time', 'service_waiters_attitude',
       'service_parking_convenience', 'service_serving_speed', 'price_level',
       'price_cost_effective', 'price_discount', 'environment_decoration',
       'environment_noise', 'environment_space', 'environment_cleaness',
       'dish_portion', 'dish_taste', 'dish_look', 'dish_recommendation',
       'others_overall_experience', 'others_willing_to_consume_again'],
      dtype='object')

In [11]:
num_classes_topic = len(tags)
num_classes_topic

20

In [12]:
set_gelu('tanh')  # 切换gelu版本

maxlen = 256
batch_size = 128
epochs = 25
dropout_rate = 0.1
num_classes_sentiment = 4

bert

In [13]:
# 建立分词器
tokenizer = Tokenizer(dict_path, do_lower_case=True)

In [14]:
def get_data(data):
    token_ids = []
    segment_ids = []
    label = []
    # 循环每个句子
    for text in tqdm(data['content'].astype(str)):
        # 分词并把token变成编号
        token_id, segment_id = tokenizer.encode(text, maxlen=maxlen)
        token_ids.append(token_id)
        segment_ids.append(segment_id)
    token_ids = sequence_padding(token_ids)
    segment_ids = sequence_padding(segment_ids)
    
    # 获取20个维度的标签
    for columns in tags:
        label.append(np.array(data[columns]).astype('uint8'))
    label = np.array(label) 
    return [token_ids, segment_ids], label

In [15]:
train_input, train_label = get_data(train_data)
valid_input, valid_label = get_data(valid_data)

100%|██████████| 15000/15000 [00:37<00:00, 404.61it/s]


In [16]:
train_label.shape

(20, 105000)

In [17]:
len(train_input)

2

In [18]:
# 加载预训练模型
bert = build_transformer_model(
    config_path=config_path,
    checkpoint_path=checkpoint_path,
    model='bert',
    return_keras_model=False,
)

pooler = Lambda(lambda x: x[:, 0], name='CLS-token')(bert.model.output)

# 多任务学习
mutil_layers = []
for i in range(num_classes_topic):
    dropout = Dropout(dropout_rate)(pooler)
    preds = Dense(num_classes_sentiment, activation='softmax', kernel_initializer=bert.initializer, name='preds_{}'.format(i))(dropout)
    mutil_layers.append(preds)
    
model = keras.models.Model(bert.model.input, mutil_layers)
# model.summary()

2021-10-25 20:15:46.168830: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:af:00.0 name: Tesla V100-PCIE-32GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 31.72GiB deviceMemoryBandwidth: 836.37GiB/s
2021-10-25 20:15:46.169317: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2021-10-25 20:15:46.169362: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2021-10-25 20:15:46.169395: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2021-10-25 20:15:46.169423: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10
2021-10-25 20:15:46.169452: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusolv

In [19]:
loss_dict = {}
loss_weights_dict = {}
for i in range(num_classes_topic):
    loss_dict['preds_{}'.format(i)] = 'sparse_categorical_crossentropy'
    loss_weights_dict['preds_{}'.format(i)] = 1.

In [20]:
# 派生为带分段线性学习率的优化器。
# 其中name参数可选，但最好填入，以区分不同的派生优化器。
# loss_weights表示每个任务的权重，可以看情况设置

AdamLR = extend_with_piecewise_linear_lr(Adam, name='AdamLR')

model.compile(loss=loss_dict,
              loss_weights=loss_weights_dict,
              optimizer=AdamLR(learning_rate=1e-4, lr_schedule={
                    int((len(train_input[0]) // batch_size * epochs) * 0.2): 1,
                    int((len(train_input[0]) // batch_size * epochs) * 0.3): 0.1
              }),
              metrics=['accuracy'],
             )

In [21]:
def adversarial_training(model, embedding_name, epsilon=1):
    """给模型添加对抗训练
    其中model是需要添加对抗训练的keras模型，embedding_name
    则是model里边Embedding层的名字。要在模型compile之后使用。
    """
    if model.train_function is None:  # 如果还没有训练函数
        model._make_train_function()  # 手动make
    old_train_function = model.train_function  # 备份旧的训练函数

    # 查找Embedding层
    for output in model.outputs:
        embedding_layer = search_layer(output, embedding_name)
        if embedding_layer is not None:
            break
    if embedding_layer is None:
        raise Exception('Embedding layer not found')

    # 求Embedding梯度
    embeddings = embedding_layer.embeddings  # Embedding矩阵
    gradients = K.gradients(model.total_loss, [embeddings])  # Embedding梯度
    gradients = K.zeros_like(embeddings) + gradients[0]  # 转为dense tensor

    # 封装为函数
    inputs = (
        model._feed_inputs + model._feed_targets + model._feed_sample_weights
    )  # 所有输入层
    embedding_gradients = K.function(
        inputs=inputs,
        outputs=[gradients],
        name='embedding_gradients',
    )  # 封装为函数

    def train_function(inputs):  # 重新定义训练函数
        grads = embedding_gradients(inputs)[0]  # Embedding梯度
        delta = epsilon * grads / (np.sqrt((grads**2).sum()) + 1e-8)  # 计算扰动
        K.set_value(embeddings, K.eval(embeddings) + delta)  # 注入扰动
        outputs = old_train_function(inputs)  # 梯度下降
        K.set_value(embeddings, K.eval(embeddings) - delta)  # 删除扰动
        return outputs

    model.train_function = train_function  # 覆盖原训练函数

In [22]:
# 写好函数后，启用对抗训练只需要一行代码
adversarial_training(model, 'Embedding-Token', 0.5)

/home/powerop/.pyenv/versions/python37/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [26]:
def evaluate(x_trues, y_trues):
    golds_topic, preds_topic = [], []
    golds_sentiment, preds_sentiment = [], []

    y_preds = np.argmax(model.predict(x_trues, batch_size = 128, verbose=1), axis=-1)
    for y_pred, y_true in zip(y_preds.T, y_trues.T):
        golds_topic.append([1 if y > 0 else 0 for y in y_true])
        preds_topic.append([1 if y > 0 else 0 for y in y_pred])
        golds_sentiment.extend(y_true)
        preds_sentiment.extend(y_pred)
    print(classification_report(y_true=golds_topic, y_pred=preds_topic, target_names=tags, digits=4))
    print(classification_report(y_true=golds_sentiment, y_pred=preds_sentiment, target_names=['未提及', '负', '中', '正'], digits=4))
    f1_score_topic = f1_score(y_true=golds_topic, y_pred=preds_topic, average='micro')
    f1_score_sentiment = f1_score(y_true=golds_sentiment, y_pred=preds_sentiment, average='micro')
    f1_score_average = float(format((f1_score_topic + f1_score_sentiment) / 2, '.4f'))
    print("f1_score_average", f1_score_average, type(f1_score_average))
    return f1_score_average

In [27]:
class Evaluator(keras.callbacks.Callback):
    """评估与保存
    """
    def __init__(self):
        self.best_val_f1 = 0.

    def on_epoch_end(self, epoch, logs=None):
        val_f1 = evaluate(valid_input, valid_label)
        if val_f1 > self.best_val_f1:
            self.best_val_f1 = val_f1
            model.save_weights('model/best_model_tag_sentiment.weights')
            model_json = model.to_json()
            with open('model/best_model_tag_sentiment.json', 'w') as json_file:
                json_file.write(model_json)
        print(
            u'val_f1: %.5f, best_val_f1: %.5f\n' %
            (val_f1, self.best_val_f1)
        )

In [28]:
evaluator = Evaluator()

model.fit(
    train_input, [train_label[i] for i in range(num_classes_topic)], 
    batch_size=batch_size,
    epochs=epochs,
    callbacks=[evaluator]
)

Epoch 1/25
15000/15000 [==============================] - 8s 556us/step


/home/powerop/.pyenv/versions/python37/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/powerop/.pyenv/versions/python37/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                          precision    recall  f1-score   support

            location_traffic_convenience     0.8863    0.8338    0.8592      3243
location_distance_from_business_district     0.7832    0.4407    0.5640      2968
                   location_easy_to_find     0.8982    0.6912    0.7812      3484
                       service_wait_time     0.0000    0.0000    0.0000      1763
                service_waiters_attitude     0.9034    0.7930    0.8446      9008
             service_parking_convenience     0.9400    0.7222    0.8168       954
                   service_serving_speed     0.8324    0.3186    0.4608      2323
                             price_level     0.8595    0.5958    0.7037      7503
                    price_cost_effective     0.7911    0.2343    0.3616      3572
                          price_discount     0.8926    0.6675    0.7638      5738
                  environment_decoration     0.8431    0.8600    0.8515      7199
               

/home/powerop/.pyenv/versions/python37/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                          precision    recall  f1-score   support

            location_traffic_convenience     0.8993    0.8369    0.8670      3243
location_distance_from_business_district     0.7556    0.5822    0.6577      2968
                   location_easy_to_find     0.8816    0.7609    0.8168      3484
                       service_wait_time     0.7699    0.2411    0.3672      1763
                service_waiters_attitude     0.9120    0.8089    0.8574      9008
             service_parking_convenience     0.9322    0.7212    0.8132       954
                   service_serving_speed     0.8981    0.5045    0.6461      2323
                             price_level     0.8853    0.7025    0.7834      7503
                    price_cost_effective     0.8335    0.4569    0.5902      3572
                          price_discount     0.8928    0.7285    0.8023      5738
                  environment_decoration     0.8738    0.8423    0.8578      7199
               

...

                                          precision    recall  f1-score   support

            location_traffic_convenience     0.8983    0.8554    0.8763      3243
location_distance_from_business_district     0.7464    0.6715    0.7070      2968
                   location_easy_to_find     0.8834    0.8042    0.8419      3484
                       service_wait_time     0.7965    0.5440    0.6464      1763
                service_waiters_attitude     0.9143    0.8223    0.8659      9008
             service_parking_convenience     0.9428    0.7778    0.8524       954
                   service_serving_speed     0.8790    0.6599    0.7539      2323
                             price_level     0.8807    0.7557    0.8134      7503
                    price_cost_effective     0.8048    0.5759    0.6713      3572
                          price_discount     0.8809    0.8099    0.8439      5738
                  environment_decoration     0.8978    0.8607    0.8789      7199
               

/home/powerop/.pyenv/versions/python37/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                          precision    recall  f1-score   support

            location_traffic_convenience     0.8945    0.8603    0.8771      3243
location_distance_from_business_district     0.7400    0.6819    0.7098      2968
                   location_easy_to_find     0.8815    0.8074    0.8428      3484
                       service_wait_time     0.7746    0.6120    0.6838      1763
                service_waiters_attitude     0.9193    0.8204    0.8670      9008
             service_parking_convenience     0.9346    0.7945    0.8589       954
                   service_serving_speed     0.8441    0.6948    0.7622      2323
                             price_level     0.8790    0.7597    0.8150      7503
                    price_cost_effective     0.7980    0.5907    0.6789      3572
                          price_discount     0.8752    0.8186    0.8459      5738
                  environment_decoration     0.9039    0.8530    0.8777      7199
               

/home/powerop/.pyenv/versions/python37/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                          precision    recall  f1-score   support

            location_traffic_convenience     0.8996    0.8566    0.8776      3243
location_distance_from_business_district     0.7600    0.6584    0.7055      2968
                   location_easy_to_find     0.8860    0.8031    0.8425      3484
                       service_wait_time     0.7720    0.6222    0.6891      1763
                service_waiters_attitude     0.9194    0.8173    0.8654      9008
             service_parking_convenience     0.9359    0.7956    0.8601       954
                   service_serving_speed     0.8498    0.6870    0.7598      2323
                             price_level     0.8768    0.7610    0.8148      7503
                    price_cost_effective     0.8007    0.5882    0.6782      3572
                          price_discount     0.8751    0.8203    0.8468      5738
                  environment_decoration     0.9011    0.8551    0.8775      7199
               

In [29]:
model.load_weights('model/best_model_tag_sentiment.weights')
print(u'final valid f1: %05f\n' % (evaluate(valid_input, valid_label)))

15000/15000 [==============================] - 8s 539us/step


/home/powerop/.pyenv/versions/python37/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                          precision    recall  f1-score   support

            location_traffic_convenience     0.8996    0.8566    0.8776      3243
location_distance_from_business_district     0.7600    0.6584    0.7055      2968
                   location_easy_to_find     0.8860    0.8031    0.8425      3484
                       service_wait_time     0.7720    0.6222    0.6891      1763
                service_waiters_attitude     0.9194    0.8173    0.8654      9008
             service_parking_convenience     0.9359    0.7956    0.8601       954
                   service_serving_speed     0.8498    0.6870    0.7598      2323
                             price_level     0.8768    0.7610    0.8148      7503
                    price_cost_effective     0.8007    0.5882    0.6782      3572
                          price_discount     0.8751    0.8203    0.8468      5738
                  environment_decoration     0.9011    0.8551    0.8775      7199
               